In [109]:
import findspark
findspark.init()

import _mssql
import re 

from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, explode, col, monotonically_increasing_id 
from pyspark.sql import Row, Column, DataFrame

In [2]:
appName = "Job description"
master  = "local" 
conf = SparkConf().setAppName(appName) \
                  .setMaster(master)
sc = SparkContext.getOrCreate(conf=conf) 
sqlContext = SQLContext(sc) 
spark = sqlContext.sparkSession
spark 

In [68]:
df = spark.read.option('header', True)\
                .option('sep', ',')\
                .option('multiLine', True) \
                .option('path', '../data/job_description.csv')\
                .option('quote', '"') \
                .option('escape', '"') \
                .format('csv')\
                .load()
df.show(n=3, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
df.select('sectors').distinct().show(truncate=False)

+------------------------------------------------------------------------------------------------+
|sectors                                                                                         |
+------------------------------------------------------------------------------------------------+
|['Kinh doanh', 'Chăm sóc khách hàng', 'Tài chính/Kế toán/Kiểm toán']                            |
|['Kinh doanh', 'Giáo dục/Đào tạo/Thư viện', 'Tài chính/Kế toán/Kiểm toán']                      |
|['Bất động sản', 'Bán hàng', 'Kinh doanh']                                                      |
|['Chăm sóc khách hàng', 'Kinh doanh', 'Bán hàng']                                               |
|['Kho vận/Vật tư/Thu mua', 'Lao động phổ thông', 'Thể dục/Thể thao']                            |
|['Thẩm định/Giám định/Quản lý chất lượng', 'Chăm sóc khách hàng', 'Ngoại ngữ']                  |
|['Quản lý điều hành', 'Kinh doanh', 'Quan hệ đối ngoại']                                        |
|['Kho vận

In [94]:
# @udf(returnType = StringType())
def to_set(sector):
    sector = sector[0]
    filters = "['\[\]]"
    r = map(lambda x: re.sub('[-&(.*)]', '/', x).replace(r'\r', '').replace(r'\n', '').replace(r'\xa0', '')\
                            .split(sep='/'), re.sub(filters, '', sector).split(sep=','))
    result = set() 
    for i in r: 
        for j in i: 
            _j = j.strip().lower()
            result.add(_j) 

    return result

industries = df.select('sectors').rdd
industries

MapPartitionsRDD[203] at javaToPython at NativeMethodAccessorImpl.java:0

In [95]:
industries = industries.flatMap(to_set).distinct().collect()

In [115]:
rdd = sc.parallelize(industries) 
id = 1
industries_rdd = rdd.map(lambda x: Row(industry = x))
industries_df = spark.createDataFrame(industries_rdd)

In [116]:
industries_df.show()

+-------------------+
|           industry|
+-------------------+
|          tài chính|
|         kinh doanh|
|            kế toán|
|chăm sóc khách hàng|
|          kiểm toán|
|           thư viện|
|            đào tạo|
|           giáo dục|
|       bất động sản|
|           bán hàng|
|             vật tư|
|            thu mua|
|           thể thao|
|            thể dục|
| lao động phổ thông|
|            kho vận|
| quản lý chất lượng|
|          ngoại ngữ|
|          thẩm định|
|          giám định|
+-------------------+
only showing top 20 rows

